# Dados sobre os clubes nesse período da Libertadores.

In [5]:
import pandas as pd


In [6]:
df = pd.read_csv("libertadores-results-ds.csv")
df.head()

,Edition,Round,Date,Home Club,Away Club,Home Score,AwayScore
0,2023,Final,4/11/2023,Fluminense FC,Boca Juniors,2,1
1,2023,Semifinal,6/10/2023,Palmeiras,Boca Juniors,1,1
2,2023,Semifinal,5/10/2023,Internacional,Fluminense FC,1,2
3,2023,Semifinal,29/09/2023,Boca Juniors,Palmeiras,0,0
4,2023,Semifinal,28/09/2023,Fluminense FC,Internacional,2,2


# Vendo a quantidade total de jogos do dataset

In [7]:
len(df)

1954

# Vendo a quantidade diferente de times no data set

In [8]:
# Contagem de times diferentes no dataset
# Um time joga no MINIMO 1 jogo em casa, por isso pode usar essa coluna
df['Home Club'].nunique()


132

# Vendo times com mais jogos

In [9]:
df['Home Club'].value_counts()

Home Club
River Plate            62
Boca Juniors           57
Nacional               54
Palmeiras              54
Atlético Mineiro       54
                       ..
Deportivo Petare        1
Everton CD              1
Deportivo Capiatá       1
Club Puebla             1
Sportivo Trinidense     1
Name: count, Length: 132, dtype: int64

# Se a Libertadores fosse pontos corridos?
### Temos esse esquema no Campeonato Paulista, onde mesmo que há a divisão dos grupos e posteriormente a fase mata-mata, há uma tabela de classificação geral
### Vou recriar essa ideia, agora na Copa Libertadores
### Será por separado por ANO

In [47]:
def statusMandante(dfAno):
    
    dfAno = dfAno.copy()  # evita warning ao criar colunas

    # Vitória mandante (True/False)
    dfAno['Vitória Casa'] = dfAno['Home Score'] > dfAno['AwayScore']

    # Empate
    dfAno['Empate'] = dfAno['Home Score'] == dfAno['AwayScore']

    # Derrota mandante 
    dfAno['Derrota Casa'] = dfAno['Home Score'] < dfAno['AwayScore']

    statusMandante = dfAno.groupby('Home Club').agg({
        'Vitória Casa': 'sum',
        'Derrota Casa': 'sum',
        'Empate': 'sum',
        'Home Score': 'sum',
        'AwayScore': 'sum',
        'Home Club': 'count'  # quantidade de jogos em casa
    }).rename(columns={
        'Vitória Casa': 'Vitórias',
        'Derrota Casa': 'Derrotas',
        'Empate': 'Empates',
        'Home Score': 'Gols Pró',
        'AwayScore': 'Gols Sofridos',
        'Home Club': 'Jogos'
    })

    return statusMandante


In [48]:
def statusVisitante(dfAno):

    dfAno = dfAno.copy()  # evita warning ao criar colunas

    dfAno['Vitória Visitante'] = dfAno['Home Score'] < dfAno['AwayScore']
    dfAno['Derrota Visitante'] = dfAno['Home Score'] > dfAno['AwayScore']
    dfAno['Empate'] = dfAno['Home Score'] == dfAno['AwayScore']

    statusVisitante = dfAno.groupby('Away Club').agg({
        'Vitória Visitante': 'sum',
        'Derrota Visitante': 'sum',
        'Empate': 'sum',
        'Home Score': 'sum',
        'AwayScore': 'sum',
        'Away Club': 'count' 
    }).rename(columns={
        'Vitória Visitante': 'Vitórias',
        'Derrota Visitante': 'Derrotas',
        'Empate': 'Empates',
        'Home Score': 'Gols Pró',
        'AwayScore': 'Gols Sofridos',
        'Away Club': 'Jogos'
    })

    return statusVisitante

In [ ]:
def criarTabelaClassificacao(df, ano):
    anoTabela = df[df['Edition'] == ano]

    tabelaMandantes = statusMandante(anoTabela)
    tabelaVisitantes = statusVisitante(anoTabela)

    # Concatena as tabelas de mandantes e visitantes para ter todos os dados em uma única tabela
    # Agrupa os dados por time (index) e soma os valores para ter o total de cada estatística
    tabelaGeral = pd.concat([tabelaMandantes, tabelaVisitantes])
    tabelaGeral = tabelaGeral.groupby(tabelaGeral.index).sum()

    tabelaGeral['Saldos Gols'] = tabelaGeral['Gols Pró'] - tabelaGeral['Gols Sofridos']
    tabelaGeral['Pontos'] = tabelaGeral['Vitórias'] * 3 + tabelaGeral['Empates']
    tabelaGeral['Aproveitamento'] = tabelaGeral['Pontos'] / (tabelaGeral['Jogos'] * 3) * 100
    tabelaGeral['Aproveitamento'] = tabelaGeral['Aproveitamento'].round(2)
    tabelaGeral = tabelaGeral.sort_values(by='Pontos', ascending=False).reset_index()
    tabelaGeral = tabelaGeral.rename(columns={'index': 'Clube'})
    
    return tabelaGeral

criarTabelaClassificacao(df, 2012)


,Clube,Vitórias,Derrotas,Empates,Gols Pró,Gols Sofridos,Jogos,Saldos Gols,Pontos,Aproveitamento
0,Corinthians,8,0,6,19,7,14,12,30,71.43
1,Boca Juniors,8,2,4,17,12,14,5,28,66.67
2,Libertad,6,2,4,21,11,12,10,22,61.11
3,Fluminense FC,6,2,2,8,9,10,-1,20,66.67
4,Santos FC,6,4,2,24,9,12,15,20,55.56
5,Universidad de Chile,5,3,4,26,8,12,18,19,52.78
6,Vélez Sarsfield,6,3,1,11,10,10,1,19,63.33
7,Vasco da Gama,5,3,2,13,10,10,3,17,56.67
8,Unión Española,4,4,2,17,13,10,4,14,46.67
9,Internacional,3,3,4,18,6,10,12,13,43.33
